In [1]:
import numpy as np
import pandas as pd

Load Mnist fafashion dataset using keras

In [2]:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


4431872/4422102 [==============================] - 0s 0us/step


In [3]:

classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']



Data preprocessing
-verify dataset 
-perform one hot encoding

In [4]:
def oneHotEncoding(y_train):
    datasetTrain=y_train.shape[0]
    classifier,count = np.unique(y_train, return_counts=True)
    y_train_classifier=np.zeros((datasetTrain,10))
    rows=0
    for i in  y_train:
        y_train_classifier[rows][i.astype('int')]=1 
        rows+=1;
    return y_train_classifier

def Normalize(x):
    return x.astype("float64") /255
    
#verify data
print('Training dataset:  '+str(x_train.shape[0]))
print('Testing dataset:  '+str(y_test.shape[0]))

#get all unique classifier
classifier,count = np.unique(y_train, return_counts=True)




#perform one hot encoding and normalization
y_train=oneHotEncoding(y_train)
x_trainNorm=Normalize(x_train)
x_testNorm=Normalize(x_test)
x_trainNorm=x_trainNorm.reshape((x_trainNorm.shape[0],x_trainNorm.shape[1]*x_trainNorm.shape[2]))
x_testNorm=x_testNorm.reshape((x_testNorm.shape[0],x_testNorm.shape[1]*x_testNorm.shape[2]))



Training dataset:  60000
Testing dataset:  10000


Define  a class for deep learning neural networks.

In [30]:
import numpy as np
class NeuralNetwork:
    def __init__(self, x, y, lr = .1,  epochs =10,HiddenLayerNuron=[60,10],activation=['relu','identity'],log=True):
        self.log=log
        self.HiddenLayerNuron=HiddenLayerNuron
        self.x = x 
        self.y = y
        self.batch= self.x.shape[0]
        self.epochs = epochs
        self.lr = lr
        self.activation=activation
        self.loss = []
        self.acc = []
        self. runAccurecy=0
        self.init_weights()
    
     
    
    def init_weights(self):
        self.W=[]
        self.b=[]
        prevInput=self.x.shape[1]
        for i in self.HiddenLayerNuron:
            self.W.append(np.random.randn(prevInput ,i))
            prevInput=i
            self.b.append(np.random.randn(i))
            
            
    def sig(self,x):
          return  1/(1+np.exp(-x))

    # Sigmoidal derivative
    def dsig(self,x):
          return self.sig(x) * (1- self.sig(x))
        
        
    def ReLU(self, x):
        #return np.maximum(0,x)
        return  np.where(x < 0, 0, x)
    
    def dReLU(self,x):
        return 1 * (x > 0) 

    def softmax(self, z):   
        z=np.exp(z)
        tmp=np.sum(z, axis = 1) 
        for i in range(z.shape[0]):       
            z[i]=z[i]/tmp[i]
        return z
    
    def feedforward(self):
        self.z=[]
        self.a=[]
        self.yHat=[]
        totalLayer=len(self.HiddenLayerNuron)
        x=self.x
        for i in range(totalLayer):
            self.z.append(x.dot(self.W[i]) + self.b[i]) 
            if(self.activation[i]=='sigmoid'):
                self.a.append(self.sig(self.z[i]))
            else:
                if(self.activation[i]=='relu'):
                    self.a.append(self.ReLU(self.z[i]))
                else:
                    self.a.append(self.z[i])
            x=self.a[i]  
        self.yHat=self.softmax(x)
        
        return self.yHat
        
        
    def backprop(self):
        totalLayer=len(self.HiddenLayerNuron)
         
        self.error = self.yHat - self.y
        dcost = (1/self.x.shape[0])*self.error
        
        self.DW=[]
        self.DB=[]
           
        
    
        
        """i=totalLayer-1
        while( i>=0):
            print("W")
            print(self.W[i].shape)
            print("b")
            print(self.b[i].shape)
            print("z")
            print(self.z[i].shape)
            print("A")
            print(self.a[i].shape)
            print("error")
            print(dcost.shape)
            i-=1"""
            
         
        """
        DW3 = np.dot(dcost.T,self.a2).T
        DW2 = np.dot((np.dot((dcost),self.W3.T) * self.dReLU(self.z2)).T,self.a1).T
        DW1 = np.dot((np.dot(np.dot((dcost),self.W3.T)*self.dReLU(self.z2),self.W2.T)*self.dReLU(self.z1)).T,self.x).T
        db3 = np.sum(dcost,axis = 0)
        db2 = np.sum(np.dot((dcost),self.W3.T) * self.dReLU(self.z2),axis = 0)
        db1 = np.sum((np.dot(np.dot((dcost),self.W3.T)*self.dReLU(self.z2),self.W2.T)*self.dReLU(self.z1)),axis = 0        
        """
        prevLayerDW=dcost
        i=totalLayer-1
        while( i>=0):
            
           
            x=[]
            
            #get input of current hidden layer
            if(i==0):
                x=self.x
            else:
                x=self.a[i-1]
            t=np.dot(x.T,prevLayerDW)
            self.DW.append(t)
            self.DB.append( np.sum(prevLayerDW,axis = 0))
            if (i>0):
                prevLayerDW=np.dot(prevLayerDW,self.W[i].T)
                 
                if(self.activation[i]=='sigmoid'):
                    tmp=prevLayerDW*self.dsig(self.z[i-1])
                    prevLayerDW=tmp
                if(self.activation[i]=='relu'):
                    tmp=prevLayerDW*self.dReLU(self.z[i-1])
                    prevLayerDW=tmp
                 

            i-=1
        
        self.DW.reverse()
        self.DB.reverse()
        
    def UpdateParam(self): 
        totalLayer=len(self.HiddenLayerNuron)
        
           
        for i in range(totalLayer):
            self.W[i] = self.W[i] - (self.lr)* self.DW[i]
            self.b[i] = self.b[i] - (self.lr)* self.DB[i]
        
    def accurecy(self,pred,y):
        y=np.argmax(y, axis=1)
        pred=np.argmax(pred, axis=1)
        tmp=pred-y
        count=np.count_nonzero(tmp == 0)
        acc=100*(count/y.shape[0])
        return acc
         
    def predict(self,x):
        self.x=x
        pred=self.feedforward()
         
        pred=np.argmax(pred, axis=1)
        return pred
    
    def shuffle(self):
        idx = [i for i in range(self.input.shape[0])]
        np.random.shuffle(idx)
        self.input = self.input[idx]
        self.target = self.target[idx]
    def RunStatistic(self):
        return self. runAccurecy 
    def train(self):
        for epoch in range(self.epochs):
            pred=self.feedforward()
          
            y=self.y
            
            acc=self.accurecy(pred,y)
            self.backprop()
            self.UpdateParam()
            self. runAccurecy=round(acc, 2)
            if(self.log):
                 print('\r steps={}/{} , Accuraacy ={}'.format((epoch+1),self.epochs,round(acc, 2)),end =" ") 
           
                
            

In [31]:

network=NeuralNetwork(x_trainNorm,y_train)
network.train()


 steps=10/10 , Accuraacy =41.08 

In [32]:
prediction=network.predict(x_testNorm)



In [33]:
prediction=y_test-prediction

accurecy=sum(x==0 for x in prediction)
accurecy=accurecy/len(prediction)
print('Test accuracy='+str(accurecy*100))

Test accuracy=44.96


In [21]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 4.0 MB/s 
     |████████████████████████████████| 143 kB 53.1 MB/s 
     |████████████████████████████████| 180 kB 55.3 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=107598f5ff2b756eb83a75054d995d63dc7f39911ebc05d190b0d256db2d009c
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [22]:
import numpy as np
import wandb
from keras.datasets import fashion_mnist
wandb.init(project="cs6910_assignment1", entity="kankan-jana")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [34]:
sweep_config = {
  "name" : "my-sweep",
  "method" : "random",
  "parameters" : {
    "epochs" : {
      "values" : [2, 5]
    },
    "lr" :{
      "min": 0.1,
      "max": 0.2
    }
  }
}
sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 7ihknbtk
Sweep URL: https://wandb.ai/kankan-jana/uncategorized/sweeps/7ihknbtk


In [35]:
def TrainModel():
  
    # read the current value of parameter "a" from wandb.config
    wandb.init()
    epochs = wandb.config.epochs
    network=NeuralNetwork(x=x_trainNorm,y=y_train,epochs=epochs,log=False)
    network.train()
    network.train()
    acc=network.RunStatistic()
    wandb.log({"epochs": epochs, "accuracy": acc})


In [ ]:
count = 5 # number of runs to execute
wandb.agent(sweep_id, function= TrainModel)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 421fwg8x with config:
wandb: 	epochs: 5
wandb: 	lr: 0.1832022118225798


accuracy,▁
epochs,▁
accuracy,37.11
epochs,5


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 8idrf5rn with config:
wandb: 	epochs: 2
wandb: 	lr: 0.11343298690219332
wandb: Currently logged in as: kankan-jana (use `wandb login --relogin` to force relogin)


accuracy,▁
epochs,▁
accuracy,16.91
epochs,2


wandb: Agent Starting Run: n84pnxvi with config:
wandb: 	epochs: 2
wandb: 	lr: 0.11665860923245566


...

accuracy,▁
epochs,▁
accuracy,42.36
epochs,5


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 0njupwd3 with config:
wandb: 	epochs: 5
wandb: 	lr: 0.1419430760008364


accuracy,▁
epochs,▁
accuracy,47.6
epochs,5


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: os1atp64 with config:
wandb: 	epochs: 2
wandb: 	lr: 0.19015843133552357


accuracy,▁
epochs,▁
accuracy,27.57
epochs,2
